In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/pytroch_study/dlwpt-code-master

/content/drive/MyDrive/Colab Notebooks/pytroch_study/dlwpt-code-master


### [4.1] 이미지 다루기

In [2]:
## 이미지 파일 로딩
import torch
import imageio
img_arr = imageio.imread('data/p1ch4/image-dog/bobby.jpg') # (720, 1280, 3) (H, W, C)
# 이미지 데이터를 다루는 파이토치 모듈은 텐서가 C x H x W 순으로 배치되어야 함
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1) # torch.Size([3, 720, 1280])
# [주의]: out은 img 저장 공간과 동일, 단순히 텐서 레벨에서 크기와 스트라이드 정보만 변경

* 신경망의 입력 (0 ~ 1), (-1 ~ 1) 사이일 때 훈련 성능이 가장 좋은 특징??
 * 빌딩 블록이 정의되는 방식이 이런 특징에 영향을 미침 >> 정확히 이해 X
* 정규화 외에 기하학적인 변환(회전, 크기 변환, 자르기) 작업도 성능에 영향

### [4.2] 3차원 이미지: 용적 데이터


* CT는 흑백 이미지처럼 하나의 밀도 채널만 있음
 * 2차원 단면을 스택처럼 쌓아 3차원 텐서를 만들면 3차원 해부도를 표현한 용적 데이터
 * N x C x D x H x W

### [4.3] 테이블 데이터 표현하기
* csv 파일 읽는 방법: 파이썬 내장 csv 모듈, 넘파이, 판다스

In [3]:
# 넘파이 활용
import numpy as np
import csv
wine_path = 'data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [4]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
print(wineq_numpy.shape, '\n', col_list)
# 넘파이 배열을 파이토치 텐서로 변환
wineq = torch.from_numpy(wineq_numpy)

(4898, 12) 
 ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


* 숫자값의 종류
 * 연속값:값 사이의 순서 O, 두 값의 차이가 의미를 지님
 * 순서값: 값 사이의 순서 O, 그 정도를 알 수 없기 때문에 두 값의 차이를 이용해서 계산 X
 * 카테고리값: 값 사이의 순서 X, 숫자적인 의미 X

In [5]:
data = wineq[:,:-1]
target = wineq[:,-1].long()
print(data.shape, target.shape)

torch.Size([4898, 11]) torch.Size([4898])


* 와인의 품질 점수
 * 정수 벡터로 처리: 값 사이에 '순서', '거리'가 있을 경우
 * 원핫인코딩: 값 사이에 '순서', '거리'가 없을 경우

In [6]:
# 원핫인코딩
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0) # unsqueeze: 싱글톤 차원 추가
print(target_onehot.shape)

torch.Size([4898, 10])


* 연속 데이터: 값을 바로 사용
* 순서 데이터: '우선순위' 있을 경우, 연속 취급/ 없을 경우, 카테고리 취급
* 카테고리 데이터: 원핫 or 임베딩 사용

In [7]:
## 데이터 가공
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
data_normalized = (data - data_mean) / torch.sqrt(data_var) # 정규화

* 임계값으로 찾기
 * 고급 인덱싱: torch.bool 데이터 타입을 사용하면 열이 True에 해당하는 행들만 접근 가능

In [8]:
# 점수가 3 이하인 열을 target에서 거르기
bad_indexes = target <= 3
print(bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum())
bad_data = data[bad_indexes]
print(bad_data.shape)

# 와인 정보를 좋음, 보통, 나쁨 카테고리로 나눠보자!
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target <7)]
good_data = data[target >= 3]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
  print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

# 이런 식으로 경향을 확인해 볼 수 있다!!

torch.Size([4898]) torch.bool tensor(20)
torch.Size([20, 11])
 0 fixed acidity          7.60   6.89   6.85
 1 volatile acidity       0.33   0.28   0.28
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   6.39
 4 chlorides              0.05   0.05   0.05
 5 free sulfur dioxide   53.33  35.42  35.31
 6 total sulfur dioxide 170.60 141.83 138.36
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.19
 9 sulphates              0.47   0.49   0.49
10 alcohol               10.34  10.26  10.51


### [4.4] 시계열 데이터 다루기
* 테이블: 모든 행은 다른 행과 독립적이었으며 행의 순서는 중요하지 않았음
* 시계열 데이터: 각 행은 연속적인 시간상에서의 한 지점을 표현


In [9]:
## 시간 차원 더하기
bikes_numpy = np.loadtxt(
    "data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,
    delimiter=',',
    skiprows=1,
    converters={1: lambda x: float(x[8:10])} # 첫 번째 열의 일자 문자열을 숫자로 변환
)
bikes = torch.from_numpy(bikes_numpy)
print(bikes.shape)

torch.Size([17520, 17])


In [10]:
## 시간 단위로 데이터 만들기
# N(샘플 수) x C(17개의 채널) x L(24, 1을 한 시간으로 취급 >> 주 단위로 할 경우, 7x24=168)

# 일별로 매 시간의 데이터셋을 구하기 위해 동일 텐서를 24시간 배치로 바라보는 뷰가 필요
print(bikes.shape, bikes.stride()) # 17520 시간에 17개 열 >> 일자, 시간, 17개 열의 세 개 축으로 변환

daily_bikes = bikes.view(-1, 24, bikes.shape[1]) # view: 텐서가 저장 공간에 있는 동일한 데이터를 다르게 바라보는 역할
print(daily_bikes.shape, daily_bikes.stride())
# C개의 채널을 가진 하루를 L시간으로 나눈 N개의 연속된 값
daily_bikes = daily_bikes.transpose(1, 2)
print(daily_bikes.shape, daily_bikes.stride()) # N x C x L

torch.Size([17520, 17]) (17, 1)
torch.Size([730, 24, 17]) (408, 17, 1)
torch.Size([730, 17, 24]) (408, 1, 17)


In [11]:
## 훈련 준비

# 우선, 첫째날의 데이터만 생각
first_day = bikes[:24].long() 
print(first_day.shape)
weather_onehot = torch.zeros(first_day.shape[0], 4)
print(first_day[:,9])

weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long() - 1,
    value=1.0
)
print(weather_onehot)
# cat 함수를 사용해 원래 데이터셋에 병합
torch.cat((bikes[:24], weather_onehot), 1)[:1]

torch.Size([24, 17])
tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])


tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [12]:
# 전체에 적용
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
print(daily_weather_onehot.shape)
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
print(daily_weather_onehot.shape)
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
print(daily_bikes.shape)

torch.Size([730, 4, 24])
torch.Size([730, 4, 24])
torch.Size([730, 21, 24])


### [4.5] 텍스트 표현하기
* 텍스트를 신경망이 처리할 수 있는 숫자의 텐서로 바꾸는 것
 * 첫번재 단계 - 데이터의 차원 정보 재정의
 * 문자 단위, 단어 단위 - 원핫인코딩 원리
* 대부분의 언어: 단어 수 >> 문자 수
 * 단어 레벨: 매우 큰 수의 클래스를 표현 + 개별 문자보다 더 많은 의미 내포함으로써 훨씬 더 많은 정보


In [13]:
## 문자 원핫인코딩: 분석 대상 텍스트에 있는 문자 집합으로 한정하여 원핫인코딩
with open('data/p1ch4/jane-austen/1342-0.txt', encoding='utf8') as f:
  text = f.read()

In [14]:
lines = text.split('\n')
line = lines[200]
print(line)

letter_t = torch.zeros(len(line), 128)
print(letter_t.shape)

for i, letter in enumerate(line.lower().strip()):
  letter_index = ord(letter) if ord(letter) < 128 else 0
  letter_t[i][letter_index] = 1

“Impossible, Mr. Bennet, impossible, when I am not acquainted with him
torch.Size([70, 128])


* 인코딩 크기를 처리할 규모로 줄이고 더 이상 늘어나지 않게 할 수 있을까?
 * 부동소수점 수를 가지는 벡터를 사용 - 임베딩
 * 단점: 의미나 문맥에 기반해 단어 사이의 거리를 두도록 배치하는 개념 포기
 * 해결책: 비슷한 맥락에서 사용한 단어들이 가까운 거리에 배치되는 임베딩을 만들어 사용
 * 흥미로운 점; 만들어진 임베딩에서 비슷한 단어들끼리 군집할 뿐 아니라 일관된 공간 관계를 유지 (사과 - 붉은 - 달콤한 + 노란 + 새콤한 = 레몬 벡터와 유사)
 